# Import

In [6]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import pathlib

AUTOTUNE = tf.data.experimental.AUTOTUNE

print("tensorflow version check : ", tf.__version__)
print("gpu check : ",tf.test.is_gpu_available())

tensorflow version check :  2.0.0
gpu check :  True


# Data processing Pipeline

## Data 불러오기
### 이미지 파일들을 그대로 불러오면 용량이 너무 크니 경로를 지정하여 다룸

In [10]:
data_dir = 'workspace/img_project/before_folder/image_train'

In [11]:
train_dir = pathlib.Path(data_dir+'/Training')
private_test_dir = pathlib.Path(data_dir+'/PrivateTest')
public_test_dir = pathlib.Path(data_dir+'/PublicTest')

In [12]:
# 각 경로 별 데이터 갯수 확인
print(len(list(train_dir.glob('*/*.jpg'))))
print(len(list(public_test_dir.glob('*/*.jpg'))))
print(len(list(private_test_dir.glob('*/*.jpg'))))

28709
3589
3589


#### dataset 만들기

In [16]:
list_ds = tf.data.Dataset.list_files(str(train_dir/'*/*'))
val_list_ds = tf.data.Dataset.list_files(str(public_test_dir/'*/*'))

In [18]:
#확인하기
for f in list_ds.take(5) : 
    print(f.numpy())

b'workspace\\img_project\\before_folder\\image_train\\Training\\Fear\\17369.jpg'
b'workspace\\img_project\\before_folder\\image_train\\Training\\Sad\\14899.jpg'
b'workspace\\img_project\\before_folder\\image_train\\Training\\Sad\\21100.jpg'
b'workspace\\img_project\\before_folder\\image_train\\Training\\Surprise\\22017.jpg'
b'workspace\\img_project\\before_folder\\image_train\\Training\\Fear\\15584.jpg'


## Data labeling을 위한 전처리

In [21]:
CLASS_NAMES = np.array([item.name for item in train_dir.glob('*')])

CLASS_NAMES

array(['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'],
      dtype='<U8')

In [24]:
CLASS_NAME_TO_ID = {}
for id, name in enumerate(CLASS_NAMES):
    CLASS_NAME_TO_ID[name] = id
    
print(CLASS_NAME_TO_ID)

{'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3, 'Neutral': 4, 'Sad': 5, 'Surprise': 6}


In [26]:
keys = list(CLASS_NAME_TO_ID.keys())
ids = list(CLASS_NAME_TO_ID.values())
print(keys,ids)

['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'] [0, 1, 2, 3, 4, 5, 6]


In [29]:
table = tf.lookup.StaticHashTable(
    initializer= tf.lookup.KeyValueTensorInitializer(
    keys= tf.constant(keys),
    values=tf.constant(ids),),
    default_value=tf.constant(-1),
    name="class_weight")

In [32]:
BATCH_SIZE = 32
IMG_HEIGHT = 96
IMG_WIDTH = 96

## data labeling을 위한 함수 선언

In [33]:
def get_label(file_path) :
    parts = tf.strings.split(file_path, '\\')
    return table.lookup(parts[-2])

In [34]:
def decode_img(img):
    img = tf.image.decode_jpeg(img,channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img,[IMG_WIDTH,IMG_HEIGHT])

In [35]:
def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

### labeling 된 데이터셋 만들기

In [39]:
labeled_ds = list_ds.shuffle(10000).map(process_path, num_parallel_calls=AUTOTUNE).batch(32)

In [43]:
for image, label in labeled_ds.take(1) :
    print("image shape : ", image.numpy().shape)
    print("Label : ", label.numpy())

image shape :  (32, 96, 96, 1)
Label :  [4 6 3 6 3 0 3 5 4 3 4 5 4 4 0 4 0 5 4 3 5 5 2 0 3 3 3 6 3 5 0 0]


# Model 만들기  

In [44]:
inputs = keras.Input(shape=(96, 96,1), name='img')

feature = tf.keras.layers.Conv2D(8, 3, padding = 'same', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3))(inputs)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(8, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(8, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.MaxPooling2D()(feature)
feature = tf.keras.layers.Dropout(0.25)(feature)


feature = tf.keras.layers.Conv2D(32, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(32, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(32, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.MaxPooling2D()(feature)
feature = tf.keras.layers.Dropout(0.25)(feature)


feature = tf.keras.layers.Conv2D(64, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(64, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(64, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.MaxPooling2D()(feature)
feature = tf.keras.layers.Dropout(0.25)(feature)

feature = tf.keras.layers.Conv2D(128, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(128, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.Conv2D(128, 3, padding='same')(feature)
feature = tf.keras.layers.BatchNormalization()(feature)
feature = tf.keras.layers.ReLU()(feature)
feature = tf.keras.layers.MaxPooling2D()(feature)
feature = tf.keras.layers.Dropout(0.25)(feature)

#feature = tf.keras.layers.Flatten()(feature)
feature = tf.keras.layers.GlobalAveragePooling2D()(feature)
feature = tf.keras.layers.Dense(256, activation='relu')(feature)
feature = tf.keras.layers.Dense(128, activation='relu')(feature)
outputs = tf.keras.layers.Dense(7, activation='softmax')(feature)

model = keras.Model(inputs=inputs, outputs=outputs)

In [46]:
model.summary()

model.compile(optimizer = tf.keras.optimizers.Adam(),
             loss = tf.keras.losses.sparse_categorical_crossentropy,
             metrics=[tf.keras.metrics.sparse_categorical_accuracy])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 96, 96, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 8)         80        
_________________________________________________________________
batch_normalization (BatchNo (None, 96, 96, 8)         32        
_________________________________________________________________
re_lu (ReLU)                 (None, 96, 96, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 8)         584       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 96, 8)         32        
_________________________________________________________________
re_lu_1 (ReLU)               (None, 96, 96, 8)         0     

In [47]:
print(len(model.trainable_variables))

print("-----------------Training Start-----------------")
model.fit(labeled_ds,epochs= 60)

54
-----------------Training Start-----------------
Epoch 1/60
898/898 [==============================] - 31s 35ms/step - loss: 1.6701 - sparse_categorical_accuracy: 0.3183
Epoch 2/60
898/898 [==============================] - 22s 24ms/step - loss: 1.3666 - sparse_categorical_accuracy: 0.4769
Epoch 3/60
898/898 [==============================] - 22s 25ms/step - loss: 1.2645 - sparse_categorical_accuracy: 0.5172
Epoch 4/60
898/898 [==============================] - 22s 25ms/step - loss: 1.1995 - sparse_categorical_accuracy: 0.5396
Epoch 5/60
898/898 [==============================] - 22s 25ms/step - loss: 1.1472 - sparse_categorical_accuracy: 0.5619
Epoch 6/60
898/898 [==============================] - 22s 24ms/step - loss: 1.1089 - sparse_categorical_accuracy: 0.5780
Epoch 7/60
898/898 [==============================] - 22s 25ms/step - loss: 1.0800 - sparse_categorical_accuracy: 0.5881
Epoch 8/60
898/898 [==============================] - 22s 25ms/step - loss: 1.0452 - sparse_categoric

# Model Test

In [49]:
val_labeled_ds = val_list_ds.map(process_path,num_parallel_calls=AUTOTUNE).batch

NameError: name 'num_parallel_calls' is not defined

In [ ]:
for i in val_labeled_ds.take(1):
  print(i[0].shape, i[1].shape)

In [ ]:
model.evaluate(val_labeled_ds)